<a href="https://colab.research.google.com/github/oneoclockc/deeplearning-for-AI/blob/main/chapter11_part02_sequence-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Processing words as a sequence: The sequence model approach

#### A first practical example

**Downloading the data**

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  41.9M      0  0:00:01  0:00:01 --:--:-- 41.8M


**Preparing the data**

In [2]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val" # val 디렉토리
train_dir = base_dir / "train" # train 디렉토리
for category in ("neg", "pos"): 
    os.makedirs(val_dir / category) # 디렉토리 안에 카테고리 생성
    files = os.listdir(train_dir / category) # train_dir에 있는 파일 가져오기
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files)) # validation sample 할당 
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname) # txt옮기기

train_ds = keras.utils.text_dataset_from_directory( # dataset 생성 
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Preparing integer sequence datasets**

In [3]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length, # 관리 가능한 입력크기 유지를 위해 처음 600개 단어만 가져온다. 
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**A sequence model built on one-hot encoded vector sequences**

In [4]:
# 원-핫 인코딩 
import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64") # 정수를 20000차원 벡터로 인코딩
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded) # 양방향 LSTM 추가 
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # 분류 계층 추가
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

**Training a first basic sequence model**

In [5]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")
# 느린 훈련 속도, 87%의 좋지 않은 성능 

Epoch 1/10
625/625 [==============================] - 178s 271ms/step - loss: 0.5157 - accuracy: 0.7596 - val_loss: 0.4956 - val_accuracy: 0.8180
Epoch 2/10
625/625 [==============================] - 170s 271ms/step - loss: 0.3495 - accuracy: 0.8767 - val_loss: 0.3001 - val_accuracy: 0.8852
Epoch 3/10
625/625 [==============================] - 169s 270ms/step - loss: 0.2683 - accuracy: 0.9042 - val_loss: 0.2798 - val_accuracy: 0.8868
Epoch 4/10
625/625 [==============================] - 169s 270ms/step - loss: 0.2267 - accuracy: 0.9225 - val_loss: 0.3074 - val_accuracy: 0.8852
Epoch 5/10
625/625 [==============================] - 169s 271ms/step - loss: 0.2036 - accuracy: 0.9309 - val_loss: 0.3449 - val_accuracy: 0.8856
Epoch 6/10
625/625 [==============================] - 170s 272ms/step - loss: 0.1847 - accuracy: 0.9373 - val_loss: 0.3096 - val_accuracy: 0.8950
Epoch 7/10
625/625 [==============================] - 169s 271ms/step - loss: 0.1675 - accuracy: 0.9441 - val_loss: 0.4191 -

#### Understanding word embeddings

전통적으로 자연어 처리에서는 단어를 의미나 발음을 무시하고 각각을 개별적인 기호로 취급한다. 단어를 벡터로 나타낼 때는 총 단어 수만큼의 길이의 벡터에서 다른 모든 값은 0으로 하고 단어 번호에 해당하는 원소만 1로 표시한다. 예를 들어 '학교', '컴퓨터', '집' 3단어만 있고 순서대로 1번, 2번, 3번이라면 학교는 
(
1
,
0
,
0
)
, 컴퓨터는 
(
0
,
1
,
0
)
, 집은 
(
0
,
0
,
1
)
로 나타내는 것이다. 이를 one-hot encoding이라고 한다. 단어 문서 행렬의 각 행, 즉 문서를 나타내는 벡터는 그 문서를 이루는 단어 벡터들을 모두 더한 것과 같다.

one-hot encoding은 단어의 의미를 전혀 고려하지 않으며 벡터의 길이가 총 단어 수가 되므로 매우 희박(sparse)한 형태가 된다는 문제가 있다. 이를 해결하기 위해 단어의 의미를 고려하여 좀 더 조밀한 차원에 단어를 벡터로 표현하는 것을 단어 임베딩(word embedding)이라고 한다.

아래 그림은 이러한 단어 임베딩의 한 가지 예시이다. 왼쪽 그림을 보면 왕과 여왕, 왕과 남자, 여왕과 여자가 같은 방향에 있다. 의미가 비슷한 단어는 비슷한 방향에 위치하게 된다. 단어 임베딩은 단어의 의미를 효과적으로 표현하기 때문에 one-hot encoding보다 학습 성능을 높일 수 있다. 또한 대량의 데이터로 단어 임베딩을 미리 학습시켜 두면, 문서 분류와 같은 과제에서 더 적은 데이터로도 학습된 임베딩을 사용하여 높은 성능을 낼 수 있다.


단어 임베딩의 종류에는 LSA, Word2Vec, GloVe, FastText 등이 있다.  

http://doc.mindscale.kr/km/unstructured/11.html

#### Learning word embeddings with the Embedding layer

**Instantiating an `Embedding` layer**

In [6]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**Model that uses an `Embedding` layer trained from scratch**

In [7]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 37s 54ms/step - loss: 0.4692 - accuracy: 0.7882 - val_loss: 0.3430 - val_accuracy: 0.8678
Epoch 2/10
625/625 [==============================] - 34s 54ms/step - loss: 0.2971 - accuracy: 0.8903 - val_loss: 0.3165 - val_accuracy: 0.8726
Epoch 3/10
625/625 [==============================] - 33s 53ms/step - loss: 0.2352 - accuracy: 0.9171 - val_loss: 0.3143 - val_accuracy: 0.8866
Epoch 4/10
625/625 [==============================] - 33s 53ms/step - loss: 0.1944 - accuracy: 0.9324 - val_loss: 0.3287 - val_accuracy: 0.8834
Epoch 5/10
625/625 [==============================] - 32s 51ms/step - loss: 0.1673 - accuracy: 0.9435 - val_loss: 0.3472 - val_accuracy: 0.8828
Epoch 6/10
625/625 [==============================] - 31s 50ms/step - loss: 0.1408 - accuracy: 0.9549 - val_loss: 0.3540 - val_accuracy: 0.8836
Epoch 7/10
625/625 [==============================] - 31s 50ms/step - loss: 0.1158 - accuracy: 0.9601 - val_loss: 0.4190 - val_accuracy:

#### Understanding padding and masking

우리는 bidirectional RNN을 사용하는데, 정방향 RNN은 토큰을 순서대로 처리하고,다른 하나는 역순으로 처리한다. 정방향 RNN은 문장이 짧을 경우 수백번의 enconde padding vector(0)를 보는 데만 시간을 소비한다 .  
이것을 무시하기 위한 방법 -> masking

**Using an `Embedding` layer with masking enabled**

In [9]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs) #  mask_zero=True
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

#### Using pretrained word embeddings

word2vec, global vectors for word representation 등 

In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2022-06-08 11:15:46--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-08 11:15:46--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-08 11:15:46--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

**Parsing the GloVe word-embeddings file**

In [12]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


**Preparing the GloVe word-embeddings matrix**

In [13]:
# 임베딩 매트릭스 구축 
# (max_words, embedding_dim) 의 형태여야 함

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary() # 이전에 구축해놓은 textvectorization layer에서 vocabulary를 받아온다
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim)) # glove vector로 채울 행렬 준비
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [14]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

**Model that uses a pretrained Embedding layer**

In [15]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________